# D-Hydro Boezemmodel V4

## Introductie

Met deze notebook bouw je een boezemmodel voor Nooderzijlvest.

Zorg dat je beschikt over een werkende Python installatie en <a href="https://github.com/openearth/delft3dfmpy#installation">D-HYDAMO omgeving</a>. Het resulterende model moet geimporteerd kunnen worden in D-HYDRO versie 0.9.7.52006 of hoger.

De verwijzing naar bestanden is geschreven in het Nederlands. De rest van de code is geschreven in het Engels. Elk code-blok is voorzien van uitleg boven het desbetreffende blok, geschreven in het Nederlands.

Download de bestanden van <a href="https://drive.google.com/drive/folders/1BXNEwlEDCtShDj5KVuHQcukup-QLTuzR">Google Drive</a> en zet deze in de folder .\beheerregister

<b>let op:</b> deze notebook werkt alleen in combinatie met een config.py. In config.py dient een variabele dimr_path te zijn opgegeven, afhankelijk van de D-Hydro installatie. Bijvoorbeeld:

dimr_path = r"c:\Program Files (x86)\Deltares\D-HYDRO Suite 1D2D (Beta) (0.9.7.52006)\plugins\DeltaShell.Dimr\kernels\x64\dimr\scripts\run_dimr.bat"

## Controle bestanden

Alle bestanden die gebruikt worden in deze tutorial staan in het code-blok hieronder. Wanneer je dit codeblok uitvoert wordt de aanwezigheid van deze bestanden gecontroleerd.



In [1]:
from pathlib import Path
from config import dimr_path
import pandas as pd

beheerregister = Path(r".\beheerregister").absolute().resolve()
excelbestanden = Path(r".\xlsx")

invoerbestanden = {"modelgebied": "Bemalings_en_lozingsgebieden.shp",
                   "branches": "Afaanvoervakken_DHYDRO_v5.shp",
                   "profielpunten": "profielpunten_3feb2021_actueel.shp",
                   "bruggen": "Bruggen_(doorstroomopeningen).shp",
                   "duikers": "duikers.shp",
                   "sluizen": "sluizen.shp",
                   "sifons": "sifons.shp",
                   "stuwen": "stuwen.shp",
                   "inlaten": "inlaten.shp",
                   "gemalen": "gemalen_v2.shp",
                   "peilgebieden": "Peilgebieden.shp"}

vorm_mapping = dict(rond=1,
                    driehoekig=1,
                    rechthoekig=3,
                    eivormig=1,
                    ellips=1,
                    Paraboolvormig=1,
                    trapeziumvormig=1,
                    heul=1,
                    muil=3,
                    langwerpig=1,
                    scherp=1,
                    onbekend=99,
                    overig=99)

for key, item in invoerbestanden.items():
    if not beheerregister.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {beheerregister.joinpath(item)}")


## Inlezen beheerregister in HyDAMO

### Aanmaken HyDAMO object

Alle benodigde modules worden geimporteerd en het HYDAMO object wordt aangemaakt bij het uitvoeren van onderstaand code-blok

In [2]:
from delft3dfmpy import DFlowFMModel, HyDAMO, Rectangular, DFlowFMWriter
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
from delft3dfmpy.datamodels.common import ExtendedGeoDataFrame
import hydrotools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

hydamo = HyDAMO(extent_file=str(beheerregister.joinpath(invoerbestanden["modelgebied"])))

### snap_to_branches
De optie om bij het inlezen te snappen naar branches en alles dat niet op boezemmodel_v4 ligt weg te gooien. Wordt nu alleen gebruikt bij duikers

In [3]:
branches = gpd.read_file(beheerregister.joinpath(invoerbestanden["branches"]))
branches.set_index("OVKIDENT", inplace=True)
branches.columns = branches.columns.str.lower()
snap_to_branches = dict(branches=branches,
                        distance=5,
                        attribute_filter={"hydromodel": "Boezemmodel_v4"}) 


### Toevoegen waterlopen

De waterlopen (branches) worden toegevoegd met volgend code-blok:
* de branches worden toegekend aan het hydamo-object
* een HyDAMO ruwheidscode (4 = manning) wordt toegekend
* de eindpunten van de branches worden gesnapped binnen een tolerantie van 1m. De coordinaten worden ook afgerond op 1m.

In [4]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["branches"]),
                           "branches",
                           attribute_filter={"hydromodel": "Boezemmodel_v4"},
                           keep_columns=["HydroModel",
                                         "bodemhoogtebovenstrooms",
                                         "bodembreedte",
                                         "taludhellinglinkerzijde",
                                         "taludhellingrechterzijde",
                                         "IWS_W_WATB"],
                           column_mapping={"OVKIDENT": "code",
                                           "IWS_AVVHOB": "bodemhoogtebovenstrooms",
                                           "AVVBODDR": "bodembreedte",
                                           "AVVTALUL": "taludhellinglinkerzijde",
                                           "AVVTALUR": "taludhellingrechterzijde"}
                          )

hydamo.branches.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.branches["ruwheidstypecode"] = 4

hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

### Aanmaken principeprofielen
Als terugvaloptie voor branches zonder profielen schrijven we principe profielen weg die we later in deze notebook gebruiken.

In [5]:
principe_profielen_df = hydrotools.get_trapeziums(gdf,
                                                  "code",
                                                  "bodembreedte",
                                                  "bodemhoogtebovenstrooms",
                                                  "iws_w_watb",
                                                  "taludhellinglinkerzijde",
                                                  "taludhellingrechterzijde")

principe_profielen_df.to_excel(r".\xlsx\principe_profielen.xlsx")

### Toevoegen yz-profielen
De yz-profielen worden ingeladen:
* De profielpunten worden geopend
* de punten worden geordend en geconverteerd naar polylinen met een xyz coordinaten
* toekennen ruwheid (manning = 35)
* lijnen langer dan 500m worden weggegooid, omdat er soms kademuren PRO_TYPE = PRO hebben gekregen
* lijnen worden toegekend aan het HyDAMO object
* lijnen die niet snappen met de branches, worden weggegooid

ToDo:
* verbeteren toekenning ruwheid

In [6]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["profielpunten"]),
                           "crosssections",
                           attribute_filter={"osmomsch_1": ["Z1"],
                                             "PRO_TYPE": "PRO"},
                           column_mapping={"PROIDENT": "code",
                                           "IWS_VOLGNR": "order",
                                           "OSMOMSCH_1": "category",
                                           "iws_hoogte": "z"},
                           z_coord=True
                           )

grouper = gdf.groupby("code")
profiles = dict()

for code, prof_gdf in grouper:
    prof_gdf = prof_gdf.sort_values("order")  #Staan ook punten met zelfde order
    first_point = prof_gdf.iloc[0]["geometry"]
    cum_dist = -1
    line = []
    for idx, (_, row) in enumerate(prof_gdf.iterrows()):
        geom = row["geometry"]
        distance = geom.distance(first_point)
        if distance > cum_dist:
            cum_dist = distance
            line += [(geom.x, geom.y, row["z"])]

    profiles[code] = [code, LineString(line)]

profiles_gdf = gpd.GeoDataFrame.from_dict(profiles,
                                          orient="index",
                                          columns=["code",
                                                   "geometry"]
                                          )

profiles_gdf["ruwheidstypecode"] = 4
profiles_gdf["ruwheidswaarde"] = 35
profiles_gdf["codegerelateerdobject"] = None

profiles_gdf = profiles_gdf.loc[profiles_gdf["geometry"].length < 500]

hydamo.crosssections.set_data(profiles_gdf,
                              index_col="code",
                              check_columns=True,
                              check_geotype=True)

hydamo.crosssections.snap_to_branch(hydamo.branches, snap_method="intersecting")
hydamo.crosssections.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen geparameteriseerde profielen

Per brug moet er een profiel worden toegevoegd

In [7]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "parametrised_profiles",
                           column_mapping={"bodemh_bov": "bodemhoogtebovenstrooms",
                                           "bodemh_ben": "bodemhoogtebenedenstrooms",
                                           "doorstroom": "bodembreedte",
                                           "hoogte_ond": "hoogteinsteeklinkerzijde",
                                           })

grouper = gdf.groupby("code")
data = {item: [] for item in ["code",
                              "bodemhoogtebovenstrooms",
                              "bodemhoogtebenedenstrooms",
                              "bodembreedte",
                              "hoogteinsteeklinkerzijde",
                              "geometry",
                              "codegerelateerdobject"]}

for code, df in grouper:
    data["code"] += [f"PRO_{code}"]
    data["codegerelateerdobject"] += [code]
    for item in ["bodemhoogtebovenstrooms",
                 "bodemhoogtebenedenstrooms",
                 "hoogteinsteeklinkerzijde",
                 "geometry"]:
        data[item] += [df.iloc[0][item]]
    data["bodembreedte"] += [df['bodembreedte'].sum()]

gdf = gpd.GeoDataFrame(data)
gdf["geometry"] = gdf.apply((lambda x: LineString([[x["geometry"].x-0.1,
                                                    x["geometry"].y-0.1],
                                                  [x["geometry"].x+0.1,
                                                   x["geometry"].y+0.1]])),
                            axis=1)

gdf["hoogteinsteekrechterzijde"] = gdf["hoogteinsteeklinkerzijde"]
gdf["ruwheidswaarde"] = 15
gdf["ruwheidstypecode"] = 4
gdf["taludhellinglinkerzijde"] = 1
gdf["taludhellingrechterzijde"] = 1

hydamo.parametrised_profiles.set_data(gdf,
                                      index_col="code",
                                      check_columns=True,
                                      check_geotype=False)

### Toevoegen bruggen
Per brug moet er een profiel worden toegevoegd

In [8]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "bridges",
                           column_mapping={"doorstro_2": "lengte",
                                           "bodemh_bov": "hoogteonderzijde"})

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "hoogteonderzijde": [frame["hoogteonderzijde"].values[0] for code,
                             frame in grouper],
        "lengte": [frame["lengte"].values[0] for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }
gdf = gpd.GeoDataFrame(data)

gdf["hoogtebovenzijde"] = gdf["hoogteonderzijde"] + 1

gdf["dwarsprofielcode"] = gdf["code"].apply(lambda x: f"PRO_{x}")
gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidstypecode"] = 4
gdf["ruwheidswaarde"] = 70

hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)
hydamo.bridges.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.bridges.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Aanmaken sluizen (als duiker)

Sluizen worden toegevoegd als hoge duikers (10m). Ontbrekende waarden worden als volgt ingevuld:
- Aantal kokers = 1
- Lengte = 20m
- Breedte = breedte principe profiel onderliggende branch
- BOK = hoogte principe profiel onderliggende branch

In [9]:
sluices_gdf = hydrotools.read_file(
    r"d:\repositories\nzv_pilot\beheerregister\Sluizen.shp",
    "culverts",
    attribute_filter={"SOORT": ["schutsluis naar een zijde",
                                "overig",
                                "schutsluis naar twee zijden",
                                "uitwateringssluis/spuisluis"]},
    keep_columns=["AANTAL_SLU", "SOORT", "branch_id"],
    column_mapping={
        "KOLKWIJDTE": "breedteopening",
        "BOS_BENEDE": "hoogtebinnenonderkantbenedenstrooms",
        "BOS_BOVEN": "hoogtebinnenonderkantbovenstrooms",
        "KOLKLENGTE": "lengte"},
    snap_to_branches=snap_to_branches
    )

sluices_gdf.set_index("code", inplace=True)
sluices_gdf.replace({'0': None, 0: None}, inplace=True)
sluices_gdf.loc[sluices_gdf["lengte"].isnull(), "lengte"] = 20
sluices_gdf.loc[sluices_gdf["aantal_slu"].isnull(), "aantal_slu"] = 1
sluices_gdf["breedteopening"] = sluices_gdf.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomwidth"] if not x["breedteopening"] else x["breedteopening"]), axis=1)

sluices_gdf["hoogtebinnenonderkantbenedenstrooms"] = sluices_gdf.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomlevel"] if not x["hoogtebinnenonderkantbenedenstrooms"] else x[
            "hoogtebinnenonderkantbenedenstrooms"]), axis=1)

sluices_gdf["hoogtebinnenonderkantbovenstrooms"] = sluices_gdf.apply(
    (lambda x: principe_profielen_df.loc[x["branch_id"]][
        "bottomlevel"] if not x["hoogtebinnenonderkantbovenstrooms"] else x[
            "hoogtebinnenonderkantbovenstrooms"]), axis=1)

sluices_gdf.loc[:, "vormcode"] = "rechthoekig"
sluices_gdf.loc[:, "hoogteopening"] = 10
sluices_gdf.reset_index(inplace=True)

### Toevoegen duikers en siphons
duikers en sifons worden samen met de sluizen als cuverts aan het HyDAMO object toegekend

ToDo
* vorm moet naar HyDAMO codering worden omgeschreven

In [10]:
culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    snap_to_branches=snap_to_branches,
                                    attribute_filter={"SOORT": ["maaipadduiker",
                                                                "tussenduiker",
                                                                "slootduiker",
                                                                "wegduiker",
                                                                "nog niet gevuld"]},
                                    column_mapping={"hoogte_a1": "hoogteopening",
                                                    "breedte": "breedteopening",
                                                    "bok_benede": "hoogtebinnenonderkantbenedenstrooms",
                                                    "bok_boven": "hoogtebinnenonderkantbovenstrooms",
                                                    "vorm": "vormcode"})

siphons_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["sifons"]),
                                   "culverts",
                                   column_mapping={"hoogte": "hoogteopening",
                                                   "breedte": "breedteopening",
                                                   "bok_benede":"hoogtebinnenonderkantbenedenstrooms",
                                                   "bok_boven":"hoogtebinnenonderkantbovenstrooms",
                                                   "vorm":"vormcode"})

gdf = gpd.GeoDataFrame(pd.concat([sluices_gdf, culverts_gdf, siphons_gdf], ignore_index=True))

gdf.loc[gdf['vormcode'].isnull(), 'vormcode'] = 'onbekend'
gdf.loc[:, 'vormcode'] = gdf.apply((lambda x: vorm_mapping[x['vormcode'].lower()]), axis=1)

gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidswaarde"] = 70
gdf["ruwheidstypecode"] = 4

hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.culverts.snap_to_branch(hydamo.branches, snap_method="ends", maxdist=5)
hydamo.culverts.dropna(axis=0, inplace=True, subset=["branch_offset"])

hydamo.culverts.loc[hydamo.culverts["lengte"].isna(), "lengte"] = 20

### Toevoegen stuwen

In [11]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["stuwen"]),
                           "weirs",
                           column_mapping={"kstident": "code",
                                           "kstsoort": "soortstuwcode",
                                           "kstkrubr": "laagstedoorstroombreedte",
                                           "kstmikho": "laagstedoorstroomhoogte",
                                           "kstregel": "soortregelbaarheidcode"})

gdf["afvoercoefficient"] = 1

hydamo.weirs.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.weirs.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=10)
hydamo.weirs.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen gemalen

In [12]:
pumps_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["gemalen"]),
                                 "pumps",
                                 column_mapping={"gecom_max_": "maximalecapaciteit"})

gemalen_gdf = pumps_gdf.copy()
pumps_gdf["codegerelateerdobject"] = pumps_gdf["code"].copy()
pumps_gdf["code"] = [f"PMP_{code}" for code in pumps_gdf["code"]]
hydamo.gemalen.set_data(gemalen_gdf,
                        index_col="code",
                        check_columns=True,
                        check_geotype=True)
hydamo.gemalen.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.gemalen.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.pumps.set_data(pumps_gdf,
                      index_col="code",
                      check_columns=True,
                      check_geotype=True)
hydamo.pumps.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.pumps.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen afsluitmiddelen

In [13]:
gdf = pd.DataFrame({"code": [],
                    "soortafsluitmiddelcode": [],
                    "codegerelateerdobject": []})

hydamo.afsluitmiddel.set_data(gdf, index_col="code")

### Toevoegen sturing

In [14]:
gpg_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]),
                                 "sturing",
                                 column_mapping={"GPGIDENT": "code",
                                                 "OPVAFWZP": "streefwaarde"
                                                 })

gpg_gdf["codegerelateerdobject"] = [code.replace("GPG", "") for code in gpg_gdf["code"]]
gpg_gdf = gpg_gdf[gpg_gdf["codegerelateerdobject"].isin(hydamo.gemalen["code"])]
con_gdf = hydamo.gemalen[~hydamo.gemalen["code"].isin(gpg_gdf["codegerelateerdobject"])]
con_gdf = gpd.GeoDataFrame(data={"code": con_gdf["code"].values,
                                 "geometry": con_gdf["geometry"].values})
con_gdf["codegerelateerdobject"] = con_gdf["code"]
con_gdf["streefwaarde"] = -999

gdf = gpd.GeoDataFrame(pd.concat([gpg_gdf, con_gdf], ignore_index=True))

gdf["bovenmarge"] = gdf["streefwaarde"] + 0.05
gdf["ondermarge"] = gdf["streefwaarde"] - 0.05
gdf["doelvariabelecode"] = 1

hydamo.sturing.set_data(gdf, index_col="code")

### Exporteren

Om het resultaat te beoordelen exporteren we alles naar shape-files. We slaan het hydamo object ook op als "pickle", zodat we bovenstaande stappen niet elke keer hoeven te herhalen

In [15]:
hydrotools.export_shapes(hydamo, path=Path(r"./hydamo_shp/boezemmodel_v4"))
hydrotools.save_model(hydamo, file_name=Path(r"./hydamo_model/boezemmodel_v4.pickle"))


## Converteren naar DFM

### Aanmaken dfm-klasse

In [16]:
dfmmodel = DFlowFMModel()

### Inlezen kunstwerken

In [17]:
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

2021-03-02 09:38:45,090 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - INFO - Currently only simple weirs can be applied. From Hydamo the attributes 'laagstedoorstroomhoogte' and 'kruinbreedte' are used to define the weir dimensions.


### Aanmaken 1D netwerk
We maken een 1D model met een rekenpuntafstand van 250m

In [18]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=250.0, seperate_structures=True)

2021-03-02 09:38:47,686 - delft3dfmpy.core.dfm - dfm - WARNING - Some structures are not linked to a branch.
2021-03-02 09:38:47,693 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000017 at: [0.0, 506.58549999999997, 522.864, 971.4883654003871], due to the structures at [-0.001, 25.63, 498.27, 514.901, 530.827, 971.4893654003871].
2021-03-02 09:38:47,696 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000026 at: [0.0, 192.933, 327.8625118166156], due to the structures at [-0.001, 190.291, 195.575, 327.8635118166156].
2021-03-02 09:38:47,703 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000088 at: [0.0, 411.5985, 1096.3735000000001, 1557.6313930450399], due to the structures at [-0.001, 281.298, 541.899, 1044.645, 1148.102, 1557.6323930450399].
2021-03-02 09:38:47,705 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000099 at: [0.0, 180.32049999999998, 355.78684147258457], due to the structures

2021-03-02 09:38:47,839 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001143 at: [0.0, 113.678, 522.4697076945143], due to the structures at [-0.001, 41.055, 186.301, 522.4707076945143].
2021-03-02 09:38:47,841 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001148 at: [0.0, 6.5035, 63.089999999999996, 135.06799999999998, 159.0447845704711], due to the structures at [-0.001, 0.1, 12.907, 113.273, 156.863, 159.0457845704711].
2021-03-02 09:38:47,845 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001172 at: [0.0, 59.0395, 175.78449999999998, 377.65748483996896], due to the structures at [-0.001, 3.223, 114.856, 236.713, 377.65848483996893].
2021-03-02 09:38:47,848 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001188 at: [0.0, 574.403, 678.9707140816221], due to the structures at [-0.001, 475.431, 673.375, 678.971714081622].
2021-03-02 09:38:47,849 - delft3dfmpy.core.dfm - dfm - INFO - Added 1

2021-03-02 09:38:47,956 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002060 at: [0.0, 605.992, 648.1133047223738], due to the structures at [-0.001, 570.499, 641.485, 648.1143047223737].
2021-03-02 09:38:47,959 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002074 at: [0.0, 121.9635, 1001.9682907369819], due to the structures at [-0.001, 5.947, 237.98, 385.566, 813.911, 1001.9692907369819].
2021-03-02 09:38:47,963 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002087 at: [0.0, 156.785, 254.356, 287.5135, 312.4995, 325.2137545184217], due to the structures at [-0.001, 78.725, 234.845, 273.867, 301.16, 323.839, 325.21475451842167].
2021-03-02 09:38:47,967 - delft3dfmpy.core.dfm - dfm - WARNING - Structures KST0338, KST1044 have the same location.
2021-03-02 09:38:47,968 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002125 at: [0.0, 11.654, 792.9720827694708], due to the structures at [-0.00

2021-03-02 09:38:48,069 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002921 at: [0.0, 631.0685000000001, 646.288539560793], due to the structures at [-0.001, 163.276, 394.842, 626.749, 635.388, 646.289539560793].
2021-03-02 09:38:48,079 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF003063 at: [0.0, 13.0165, 512.1186048974606], due to the structures at [-0.001, 0.1, 25.933, 512.1196048974606].
2021-03-02 09:38:48,082 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF003096 at: [0.0, 254.69549999999998, 283.764, 825.8558171432509], due to the structures at [-0.001, 241.496, 267.895, 299.633, 798.196, 825.8568171432509].
2021-03-02 09:38:48,084 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF003106 at: [0.0, 109.025, 436.15564302691666], due to the structures at [-0.001, 43.715, 174.335, 436.15664302691664].
2021-03-02 09:38:48,089 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on bra

2021-03-02 09:38:48,188 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF003946 at: [0.0, 236.11849999999998, 349.1514302847546], due to the structures at [-0.001, 131.945, 340.292, 349.1524302847546].
2021-03-02 09:38:48,192 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF003991 at: [0.0, 935.0550000000001, 1540.3980000000001, 4128.365679052618], due to the structures at [-0.001, 296.178, 925.164, 944.946, 1453.006, 1627.79, 1814.113, 2725.914, 4128.366679052619].
2021-03-02 09:38:48,196 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004038 at: [0.0, 573.4404999999999, 628.7135000000001, 754.627, 893.9934348935511], due to the structures at [-0.001, 25.946, 518.982, 627.899, 629.528, 879.726, 893.9944348935511].
2021-03-02 09:38:48,199 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004066 at: [0.0, 51.1745, 107.4935096705922], due to the structures at [-0.001, 0.146, 102.203, 107.4945096705922

2021-03-02 09:38:48,319 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005017 at: [0.0, 282.7685, 872.3530646976778], due to the structures at [-0.001, 278.533, 287.004, 872.2530646976778, 872.3540646976778].
2021-03-02 09:38:48,322 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005035 at: [0.0, 29.4635, 44.882999999999996, 54.31888419066479, 55.815768381329576], due to the structures at [-0.001, 22.083, 36.844, 52.922, 55.715768381329575, 55.816768381329574].
2021-03-02 09:38:48,326 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005072 at: [0.0, 339.61181705967755, 396.6436341193551], due to the structures at [-0.001, 282.68, 396.5436341193551, 396.6446341193551].
2021-03-02 09:38:48,332 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005096 at: [0.0, 91.02, 300.01608821790177], due to the structures at [-0.001, 0.1, 181.94, 300.01708821790174].
2021-03-02 09:38:48,334 - delft3dfmpy.core.dfm

2021-03-02 09:38:48,452 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006025 at: [0.0, 32.271, 890.068272738783], due to the structures at [-0.001, 27.062, 37.48, 881.01, 890.069272738783].
2021-03-02 09:38:48,456 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006033 at: [0.0, 141.1520742160941, 268.7771484321882], due to the structures at [-0.001, 13.627, 268.6771484321882, 268.7781484321882].
2021-03-02 09:38:48,462 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006182 at: [0.0, 230.40300000000002, 303.11449671255025], due to the structures at [-0.001, 171.708, 289.098, 303.1154967125502].
2021-03-02 09:38:48,465 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006297 at: [0.0, 11.8505, 253.88070148160438], due to the structures at [-0.001, 1.82, 21.881, 253.88170148160438].
2021-03-02 09:38:48,467 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006300 at: [0.0, 3.379

2021-03-02 09:38:48,592 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010586 at: [0.0, 69.2065, 86.94257733502819], due to the structures at [-0.001, 62.449, 75.964, 86.9435773350282].
2021-03-02 09:38:48,594 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010588 at: [0.0, 77.172, 128.3048114435671], due to the structures at [-0.001, 56.76, 97.584, 128.3058114435671].
2021-03-02 09:38:48,600 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010593 at: [0.0, 520.617, 617.4527064596491], due to the structures at [-0.001, 494.577, 546.657, 617.4537064596491].
2021-03-02 09:38:48,614 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF016421 at: [0.0, 75.19, 150.362, 158.41414240875025], due to the structures at [-0.001, 1.082, 149.298, 151.426, 158.41514240875026].
2021-03-02 09:38:48,632 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF016490 at: [0.0, 17.044, 25.366999999999997,

### Toevoegen cross-sections

We kennen profielen toe, waar we profielen missen vullen we deze aan met principeprofielen.

In [19]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} branches are still missing a cross section.")
print(f"{len(dfmmodel.crosssections.get_structures_without_crosssection())} structures are still missing a cross section.")

if len(dfmmodel.crosssections.get_branches_without_crosssection()) > 0:
    print("adding trapezium profiles on branches with missing crosssections.")
    dfmmodel = hydrotools.add_trapeziums(dfmmodel, principe_profielen_df)

2021-03-02 09:38:50,841 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of branches without cross section is: 2385.
2021-03-02 09:38:59,261 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 863.
No parametrised crossections available for branches.
2021-03-02 09:38:59,268 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of structures without cross section is: 636.
2021-03-02 09:38:59,294 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 636.
2021-03-02 09:38:59,346 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'normgeparametriseerd' the number of structures without cross section is: 0.
863 branches are still missing a cross section.
0 structures are still missing a cross section.
adding trapezium profiles on branches with missing crosssections.


### Toevoegen randvoorwaarden

In [20]:
rvw_df = pd.read_excel(excelbestanden.joinpath("randvoorwaarden.xlsx"))

for _,row in rvw_df.iterrows():
    branch_id = row["BRANCH_ID"]
    pt = hydamo.branches.loc[branch_id]["geometry"].coords[int(row["COORD"])]
    series = pd.Series(data=[0.0, 0.0],
                       index=[pd.Timestamp("2000-01-01"),
                              pd.Timestamp("2100-01-01")]
                       )
    dfmmodel.external_forcings.add_boundary_condition(name=f"BND_{branch_id}",
                                                      pt=pt,
                                                      bctype=row["TYPE"],
                                                      series=series)

### Wegschrijven model

In [21]:
dfmmodel.mdu_parameters["refdate"] = 20000101
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 0.5 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "120. 0. 0."
dfmmodel.mdu_parameters["cflmax"] = 0.7

dfmmodel.dimr_path = dimr_path
fm_writer = DFlowFMWriter(dfmmodel, output_dir=r"modellen\boezemmodel", name="boezemmodel")

fm_writer.objects_to_ldb()
fm_writer.write_all()

if Path(r"modellen\boezemmodel\fm\boezemmodel.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

Model is weggeschreven


## Importeren in de D-Hydro suite

Open het model nu in D-Hydro:
1. Open een "Empty Project"
2. In de "Home" ribbon, ga naar "Import" en selecteer "Flow Flexible Mesh Model"
3. Selecteer het bestand "boezemmodel.mdu" in ".\dfm_model\fm\boezemmodel.mdu"
4. Wacht tot het model is geimporteerd.....

Het geimporteerde model is nu zichtbaar in de D-Hydro suite
<img src="png/boezemmodel.png"/>